
# Clustering image into sementic labels

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## Installing the Latest Tensorflow library 
---
<p>tensorflow version should be 2.11.0 </p>




In [ ]:
# for loading/processing the images  
from tensorflow.keras.preprocessing.image import load_img 
from tensorflow.keras.preprocessing.image import img_to_array 
from IPython.display import Markdown as md
from tensorflow.keras.applications.convnext import preprocess_input
from sklearn import metrics
import tensorflow as tf
# models 
from sklearn.cluster import Birch
from tensorflow.keras.applications.convnext import ConvNeXtXLarge
from keras.models import Model
from sklearn.cluster import AgglomerativeClustering
# clustering and dimension reduction
from yellowbrick.cluster.elbow import kelbow_visualizer
from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans as MKMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.manifold import TSNE
# for everything else
import os
import numpy as np
import matplotlib as mtp
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle
# images are location
path = r"/content/drive/MyDrive/STL10"

# os.chdir(path)
filess=[]

# # this list holds all the image filename
# # creates a ScandirIterator aliased as files
# with os.scandir(path) as files:
#   # loops through each file in the directory
#     for file in files:
#       filess.append(file.name) 
# #     print(i)        
# # filess = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]  

# print("Number of files :",len(filess))  
             


In [ ]:
import shutil
destination='/content/drive/MyDrive/CombineFolder'
i=0
for file in filess:
  print(i)
  shutil.copy2(os.path.join(path, file),destination)
  i=i+1





In [ ]:
!python3 -m venv .env

In [ ]:
model =ConvNeXtXLarge()
#model.summary()

In [ ]:
model =ConvNeXtXLarge(include_top=False)
model = Model(inputs = model.inputs, outputs = tf.keras.layers.GlobalAveragePooling2D())
model.summary()

In [ ]:
%%time
import traceback
def extract_features(file, model):

    img = load_img(file, target_size=(224,224))

    img = img_to_array (img) 

    reshaped_img = img.reshape(1,224,224,3) 

    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features 
data = {}
p = r"/content/drive/MyDrive/unsplash-images-collection"

 # lop through each image in the dataset
for files in filess:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(files,model)
        data[files] = feat
    # if something fails, print erro
    except Exception:

      traceback.print_exc()
# f=open('/content/drive/MyDrive/BatchThreeData.sav', 'wb')
# pickle.dump(data,f)
# f.close()        
 


In [ ]:
import plotly.express as px

In [ ]:
data=pickle.load(open('/content/drive/MyDrive/fcovext.sav', 'rb'))

In [ ]:
# SavedDataOne=pickle.load(open('/content/drive/MyDrive/BatchOneData.sav', 'rb'))
# SavedDataTwo=pickle.load(open('/content/drive/MyDrive/BatchTwoData.sav', 'rb'))
# SavedDataThree=pickle.load(open('/content/drive/MyDrive/BatchThreeData.sav', 'rb'))
# SavedDataFour=pickle.load(open('/content/drive/MyDrive/BatchFourData.sav', 'rb'))

In [ ]:

# len(SavedData)


In [ ]:
# len(set(SavedData.keys()))


  

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [ ]:
#get a list of the filenames
filenames = np.array(list(data.keys()))
# get a list of just the features
feat = np.array(list(data.values()))
print(feat.shape)
# reshape so that there are 210 samples of 1280 4096 4032 1536 1920 vectors
feat = feat.reshape(-1,2048)
# reduce the amount of dimensions in the feature vector
pca =TSNE(n_components=3, random_state=54)
pca.fit(feat)
x = pca.fit_transform(feat)

In [ ]:
reducedDimensionDF=pd.read_csv('/content/drive/MyDrive/3dplotfeatmeusum.csv')

In [ ]:
reducedDimensionDF=pd.DataFrame(x)

In [ ]:
df=pd.DataFrame(np.array([]))

In [ ]:
reducedDimensionDF['0']

In [ ]:
df['0']=reducedDimensionDF[0]
df['1']=reducedDimensionDF[1]
df['2']=reducedDimensionDF[2]
df['c']=reducedDimensionDF['c']


In [ ]:
!pip install ipympl

In [ ]:
%matplotlib widget
plt.figure(figsize=(10, 10))
ax = plt.axes(projection='3d')
colors=["#e60049", "#0bb4ff", "#53e991", "#e6d800", "#9b19f5", "#ffa300", "#dc0ab4", "#b3d4ff", "#00bfa0", "#991f17"]
ax.scatter3D(reducedDimensionDF[0], reducedDimensionDF[1], reducedDimensionDF[2], c=kmeans.labels_, cmap=mtp.colors.ListedColormap(colors),s=30);
plt.xlabel("X")
plt.ylabel('Y');



In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
plt.figure(figsize=(10, 10))
plt.xticks(ticks=(-1,15))
ax = plt.axes(projection='3d')
colors=["#e60049", "#0bb4ff", "#53e991", "#e6d800", "#9b19f5", "#ffa300", "#dc0ab4", "#b3d4ff", "#00bfa0", "#991f17"]

reducedDimensionDF=pd.DataFrame(x)
ax.scatter3D(reducedDimensionDF[0], reducedDimensionDF[1], reducedDimensionDF[2], c=kmeans.labels_, cmap=mtp.colors.ListedColormap(colors));

In [ ]:
reducedDimensionDF=reducedDimensionDF.drop(reducedDimensionDF[1].idxmax())

In [ ]:
reducedDimensionDF[1].idxmax()

In [ ]:
reducedDimensionDF['c']=kmeans.labels_

In [ ]:
str(kmeans.labels_)

In [ ]:

# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse)
plt.xlabel(r'Number of clusters *k*')
plt.ylabel('silhouette score');
        

In [ ]:

km=kelbow_visualizer(Birch(), x, k=(1,20))
k=km.elbow_value_ 
metrics.silhouette_score(x, km.labels_, metric='euclidean')

In [ ]:
print(k)

In [ ]:
featureDF=pd.DataFrame(x)


In [ ]:
# featureDF["cluster_id"]=kmeans.labels_

In [ ]:
featureDF.to_csv('/content/drive/MyDrive/MuseumImagesFeature.csv', sep='\t', index=True,header=True)

# K mean clustering


In [ ]:
len(groups)

In [ ]:
#cluster feature vectors

kmeans=Birch(n_clusters=10).fit(x)

# # kmeans=DBSCAN(eps=3.8, min_samples=2).fit(x)
# # holds the cluster id and the images { id: [images] }
# groups = {}
# for file, cluster in zip(filenames,kmeans.labels_):
#     if cluster not in groups.keys():
#         groups[cluster] = []
#         groups[cluster].append(file)
#     else:
#         groups[cluster].append(file)
# # function that lets you view a cluster (based on identifier)   


In [ ]:
df.head()

In [ ]:
import plotly.express as px

fig = px.scatter_3d(df[:1000], x='0', y='1', z='2',color='c',)
fig.show()

## View cluster

In [ ]:
def view_cluster(cluster,no):
    plt.figure(figsize = (20,20));
    
    # gets the list of filenames for a cluster
    files = groups[cluster]
    # only allow up to 30 images to be shown at a tim

    files = files[:no]
      
    # plot each image in the cluster
    for index, file in enumerate(files):
      plt.subplot(10,10,index+1);
      img = load_img("/content/drive/MyDrive/CombineFolder/"+str(file))
      img = np.array(img)
      plt.imshow(img)
      plt.axis('off')
      


In [ ]:
print(filenames[2942])
img = load_img("/content/drive/MyDrive/imageBackup/var/www/museum/upload_museum/thumb_aa988447067c0355df0223065af9d055.jpg")
img = np.array(img)
plt.imshow(img)
plt.show()

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
featuresExtracted = x

neigh = KNeighborsRegressor(n_neighbors=20).fit(featuresExtracted,kmeans.labels_)
arrayOfNeigh = neigh.kneighbors([x[0]], return_distance=False)

print("20 nearest neighbours are :" ,arrayOfNeigh)






In [ ]:
def replacelabel(a,arr):
  for i in range(len(a)):
      if(a[i]==0):
        a[i]=arr[0]
      elif(a[i]==1):
        a[i]=arr[1]
      elif(a[i]==2):
        a[i]=arr[2]
      elif(a[i]==3):
        a[i]=arr[3]
      elif(a[i]==4):
        a[i]=arr[4] 
      elif(a[i]==5):
        a[i]=arr[5]
      elif(a[i]==6):
        a[i]=arr[6]
      elif(a[i]==7):
        a[i]=arr[7]
      elif(a[i]==8):
        a[i]=arr[8]
      elif(a[i]==9):
        a[i]=arr[9]                 
  return a 

In [ ]:
for i in range(44):
  view_cluster(i,10)

In [ ]:
view_cluster(8,50)

In [ ]:
# airplane 0
# bird 1 
# car 2
# cat 3
# deer 4
# dog 5
# horse 6
# monkey 7
# ship 8
# truck 9
u=['0','1','2','3','4','5','6','7','8','9','10','11','12','13,']
c=replacelabel(kmeans.labels_,u)

print(c)

In [ ]:
labels=pd.read_csv('/content/drive/MyDrive/Label-main.csv')
print(labels['label'])
fname=np.array(labels['filename'])
labels=np.array(labels['label'])
arr=[]
for i in range(len(labels)):
  for j in range(len(labels)):

    if(str(filenames[i])==str(fname[j])):
      arr.append(labels[j])
arr=np.array(arr)

In [ ]:
arr=np.array(arr)-1

In [ ]:
accy=metrics.accuracy_score(arr,c)

In [ ]:
md(f'# Accuracy of the model : {accy*100}')

# 2d Graph 

In [ ]:
# from sklearn.manifold import TSNE
# pca = TSNE(n_components=2, random_state=22)
# pca.fit(feat)
# x = pca.fit_transform(feat)
kmeans =Birch(n_clusters=15)
kmeans.fit(x)


In [ ]:
y= pd.DataFrame(x)

In [ ]:
plt.figure(figsize=(8, 8))
colors = ['#1f77b4', '#aec7e8', '#ff7f0e', '#ffbb78', '#2ca02c',          '#98df8a', '#d62728', '#ff9896', '#9467bd', '#c5b0d5',          '#8c564b', '#c49c94', '#e377c2', '#f7b6d2', '#7f7f7f',          '#c7c7c7', '#bcbd22', '#dbdb8d', '#17becf', '#9edae5',          '#393b79', '#5254a3', '#6b6ecf', '#9c9ede', '#637939']
plt.scatter(y[0],y[1],c=kmeans.labels_,linewidths=0.2,cmap=mtp.colors.ListedColormap(colors),s=8)  
plt.xticks(())
plt.yticks(())
plt.show()

In [ ]:
a=np.array(arr).reshape(-1,2000)

In [ ]:
a.shape